In [1]:
# Lectura del archivo .csv de training
import pyspark_csv as pycsv
sc.addPyFile('pyspark_csv.py')
plaintext_rdd = sc.textFile('train.csv')
dataframe = pycsv.csvToDataFrame(sqlCtx, plaintext_rdd, parseDate=False)

data = dataframe.rdd
data = data.sample(False, 0.001)
#data.count()

In [ ]:
from bs4 import BeautifulSoup # Para eliminar tags html
import re # Expresiones regulares para eliminar puntuacion
from nltk.corpus import stopwords # Stopwords para eliminar palabras comunes
from nltk.stem.lancaster import LancasterStemmer

def aplicarStemming(x):
    words = x.split()
    st = LancasterStemmer()
    new_words = []
    for w in words:
        new_words.append(st.stem(w))
    return " ".join(new_words)

def borrarPalabrasComunes(x):
    words = x.split()
    stop_words = set(stopwords.words("english"))
    stop_words.remove("not")
    new_words = []
    for w in words:
        if(not w in stop_words):
            new_words.append(w)
    return " ".join(new_words)

def borrarSimbolos(x):
    aBorrar = ",@#$-.():[]!?"
    for c in aBorrar:
        x = x.replace(c, " ")
    return x

def considerarEmoticonesPuntuacion(x):
    # Lista de caritas felices
    caras_felices = [":)", "(:", "[:", ":]", "c:", "=)", "=]", "(=", "[=", "c=",
                    "=D", ":D", ";)", "(;", ";D"]
    for emoji in caras_felices:
        x = x.replace(emoji , "SMILING_FACE")
    # Lista de caritas tristes
    caras_tristes = [":(", ":[", "):", "]:", ":c", "=(", "=[", "]=", "=c", "D=", 
                    "D:", ";(", ");", "D;", ]
    for emoji in caras_tristes:
        x = x.replace(emoji, "SAD_FACE")
    # Lista de caritas sorprendidas
    caras_sorpr = [":0", ":o", "0:", "o:", "=o", "0="]
    for emoji in caras_sorpr:
        x = x.replace(emoji, "SURPRISED_FACE") 
    # Puntuación (signos ! y ?)
    x = x.replace("!!!", " ADMIRx3")
    x = x.replace("!!", " ADMIRx2")
    x = x.replace("???", " QUESx3")
    x = x.replace("??", " QUESx2")
    x = x.replace("?!", " ADM_QUES")
    x = x.replace("!?", " ADM_QUES")
    x = x.replace("!", " ADMIRx1")
    x = x.replace("?", " QUESx1")
    return x

# Función encargada de realizar un pre-procesamiento de los textos de las reviews
# según lo considerado por nuestro diseño del TP. Para ello, se recibe el set de
# entrenamiento como un RDD de reviews, que son tuplas (texto, puntaje).
# Las distintas acciones que la función realiza sobre el texto de las reviews
# dependen de los flags de procesamiento recibidos en flagsP (como lista).
# A continuación la lista de acciones controlada por cada flag de flagsP:
# flagsP[0] controla la eliminación de palabras comunes ("a", "the", "of", etc.)
# fragsP[1] elimina las palabras de frecuencia menor a *un número*
# flagsP[2] activa el uso de stemming sobre las palabras de la review
# flagsP[3] activa el reconocimiento de emoticones y puntuaciones ?,!
# continuar...
# Acciones que el pre-procesador de reviews hace siempre:
# - Eliminar tags html
# - Convertir todo a minúsculas
# - Eliminar los siguientes símbolos: "," "@" "#" "$" "-" "." "(" ")" ":" "]" "["
# (En el caso de considerar emoticones o puntuación no lo hace hasta después de
# detectar todos los emoticones o símbolos deseados correspondientes)
def preprocesar_reviews(elSet, flagsP):
    nuevoSet = elSet.map(lambda x: (BeautifulSoup(x[0], "lxml").getText(), x[1]) )
    nuevoSet = nuevoSet = nuevoSet.map(lambda x: (x[0].lower(), x[1]))
    
    if(flagsP[0]):
        nuevoSet = nuevoSet.map(lambda x: (borrarPalabrasComunes(x[0]), x[1]))
    
    if(flagsP[2]):
        nuevoSet = nuevoSet.map(lambda x: (aplicarStemming(x[0]), x[1]))
    
    if(flagsP[2]):
        nuevoSet = nuevoSet.map(lambda x: (considerarEmoticonesPuntuacion(x[0]), x[1]))
    nuevoSet = nuevoSet = nuevoSet.map(lambda x: (borrarSimbolos(x[0]), x[1]))
    
    return nuevoSet


In [2]:
# Todo este bloque define la realización del k-fold crossed validation.
# El método funciona así: recibe un set de entrenamiento y hace sobre el
# mismo la técnica de k-fold crossed validation. El formato del set debe
# ser un RDD de TUPLAS de la forma: (features, categoria) donde la clave
# features puede ser cualquier basura, y categoria es el valor numérico
# que se desea predecir (aka el puntaje de cada review). En cada pasada
# del k-fold crossed validation, se invocan a las funciones de entrenar
# func_entrenar y a las de predicción func_predecir. Éstas dos funciones
# deben trabajar de manera global con el/los compresor/es o el SVM. Sus
# firmas deben ser las siguientes:
# func_entrenar recibe un set de entrenamiento (en el mismo formato que
# el set original, como tuplas feature,cat.) y prepara al compresor o SVM
# para las predicciones usando ese set.
# func_predecir recibe un set a predecir (en el mismo formato de tuplas
# feature, cat) y debe devolver OTRO set (también en el mismo formato!)
# que correspondan a las predicciones hechas por el SVM o compresores.
# Observación importante: como el k-fold crossed validation en sí no
# tiene ni idea qué usamos para predecir, todo lo demás ajeno a eso,
# incluyendo la selección de hiperparametros, debe hacerse "por fuera",
# ya sea con un pre-procesamiento de las reviews o en la función de entrenar.

def fooCount(x):
    global contadora
    contadora += 1
    return (x, contadora)

def calculo_ECM(predSet, valSet):
    #cant = predSet.count()
    cant = 1
    setAux = predSet.union(valSet)
   # setAux = setAux.map(lambda x: (np.array_str(x[0]), x[1]) )
    setAux = setAux.reduceByKey(lambda x,y: float(x)-float(y)).map(lambda x: x[1]*x[1])
    ecm = setAux.reduce(lambda x,y: x+y)
    return (ecm/float(cant))

def k_fold_crossed_validation(elSet, func_entrenar, func_predecir):
    cantParticiones = 8
    ecm_acum = 0.0
    largoSet = elSet.count()
    largoParticion = largoSet / cantParticiones
    setauxi = elSet.map(fooCount)
    for j in range (1, cantParticiones+1):
        # Obtengo el testSet como la particion j-ésima y el trainSet como
        # todo el resto del set recibido menos el testSet
        #print "El set:", elSet.take(2)
        testSet = setauxi.filter(lambda x: (x[1] % cantParticiones) == (j-1)).map(lambda x: x[0])
        trainSet = setauxi.filter(lambda x: (x[1] % cantParticiones) != (j-1)).map(lambda x: x[0])
        #print "Antes de entrenar:", trainSet.take(2)
        # Entreno contra trainSet
        func_entrenar(trainSet)
        # Testeo contra testSet
        setResultados = func_predecir(testSet)
        ecm_acum += calculo_ECM(setResultados, testSet)
        print "ECM acumulado iteracion", j, "es:", ecm_acum
    # Obtengo el ECM promedio de la validación
    print "ECM promedio:", (ecm_acum/float(cantParticiones))
    
    
contadora = 0 # Se usa, no tocar

In [3]:
from ppmc import *
import numpy as np
import gc
def crear_compresor(x):
    aux = CompresorHibrido(5)
    aux.entrenar(x)
    return aux

def combinar_compresor(x, y):
    x.combinarTabla(y.verTablas())
    return x

def entrenar_comp(trainSet):
    global comp
    comp = []
    gc.collect()
    for i in range(1, 6): # 5 puntajes posibles
        rev_act = trainSet.filter(lambda x: x[1] == i).map(lambda x: x[0])
        tmp = CompresorHibrido(5)
        tmp.entrenar(' ')
        
        aux2 = rev_act.map(lambda x: crear_compresor(x)).reduce(lambda x,y: combinar_compresor(x, y))
        
        tmp.combinarTabla(aux2.verTablas())
        
        comp.append(tmp)           
    return

def asignar_puntuacion(texto):
    global comp
    minimo = -1
    pred = 0
    for i in range(0, 5):
        act = comp[i].ratioCompresion(texto)
        if ((minimo == -1) or (act < minimo)):
            minimo = act
            pred = i+1
    return pred

def predecir_comp(testSet):
    setResultados = testSet.map(lambda x: (x[0], asignar_puntuacion(x[0])))
    return setResultados

comp = []

# Primero guardo las reviews en formato (texto, puntaje) y pre-proceso
reviews = data.map(lambda x: (x.Text, x.Prediction))

# Hago k-fold crossed validation contra las reviews
k_fold_crossed_validation(reviews, entrenar_comp, predecir_comp)


ECM acumulado iteracion 1 es: 110.0


KeyboardInterrupt: 

In [ ]:
qwe = data.map(lambda x: x['Text'])
qwe.take(1)
comp = []
comp

In [ ]:
reviews = data.map(lambda x: (x.Text, x.Prediction))
rev_act = reviews.filter(lambda x: x[1] == 2).map(lambda x: x[0])
rev_act = rev_act.map(lambda x: crear_compresor(x)).reduce(lambda x, y: combinar_compresor(x,y))

In [ ]:
data.count()

In [ ]:
data.take(2)